In [1]:
import time

#### Decorator to capture Run Time

In [2]:
def runtime_decorator(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        ret = func(*args)
        end = time.time()
        print(f"the Function {func.__name__} took {end - start} seconds to run")
        return ret
    return wrapper



In [3]:
@runtime_decorator
def generate_numbers(n):
    return [i for i in range(n)]

In [4]:
l = generate_numbers(10_000_000)
len(l)

the Function generate_numbers took 0.4804706573486328 seconds to run


10000000

#### Decorator to Sum even numbers

In [5]:
def add_up(func):
    def wrapper(*args):
        return sum(func(*args))
    
    return wrapper

In [6]:
@add_up
def evens(n):
    return [2 * i for i in range(n)]

In [7]:
evens(5)

20

#### Generic Decorator

In [6]:
from functools import wraps

def decorator_factory(*d_args, **d_kwargs):
    print(d_args)
    def decorator(func):        
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Do something before
            value = func(*args, **kwargs)
            # Do something after; potentially modify value
            return value

        return wrapper
    return decorator



In [7]:
@decorator_factory(1, 2)
def func(*args, **kwargs):
    pass

(1, 2)


#### Generic Decorator for all class methods

In [15]:
def runtime_decorator(original_function):      
    print("decorating")
    def new_function(*args,**kwargs):
        print("starting timer")
        import datetime                 
        before = datetime.datetime.now()                     
        x = original_function(*args,**kwargs)                
        after = datetime.datetime.now()                      
        print("Elapsed Time = {0}".format(after-before))
        return x                                             
    return new_function

def time_all_class_methods(Cls):
    class NewCls(object):
        def __init__(self,*args,**kwargs):
            self.oInstance = Cls(*args,**kwargs)
        def __getattribute__(self,s):
            """
            this is called whenever any attribute of a NewCls object is accessed. This function first tries to 
            get the attribute off NewCls. If it fails then it tries to fetch the attribute from self.oInstance (an
            instance of the decorated class). If it manages to fetch the attribute from self.oInstance, and 
            the attribute is an instance method then `time_this` is applied.
            """
            try:    
                x = super(NewCls,self).__getattribute__(s)
            except AttributeError:      
                pass
            else:
                return x
            x = self.oInstance.__getattribute__(s)
            if type(x) == type(self.__init__): # it is an instance method
                return runtime_decorator(x)    # this is equivalent of just decorating the method with runtime_decorator
            else:
                return x
    return NewCls

In [16]:
@time_all_class_methods
class Foo(object):
    def a(self):
        print("entering a")
        import time
        time.sleep(3)
        print("exiting a")

oF = Foo()
oF.a()

decorating
starting timer
entering a
exiting a
Elapsed Time = 0:00:03.015947


#### https://www.codementor.io/@sheena/advanced-use-python-decorators-class-function-du107nxsv
#### https://pythonguide.readthedocs.io/en/latest/python/decorator.html